In [14]:
import duckdb
import polars as pl
import altair as alt
import pandas as pd
import pyarrow as pa

# Play-around code to get a sense of Duckdb, polars, and altair

In [3]:
con = duckdb.connect(database='/media/shortterm_ssd/Clay/databases/duckdb/rcs-db.duckdb')

In [3]:
con.execute("CREATE SCHEMA overnight;")

In [4]:
con.execute("CREATE TABLE overnight.r02L AS SELECT * FROM '/media/shortterm_ssd/Clay/databases/Sleep_10day_with_autonomic/RCS02L/Overnight/*.parquet'")

In [88]:
con.execute("DESCRIBE overnight.r02L")
con.fetchall()

[('localTime', 'TIMESTAMP', 'YES', None, None, None),
 ('DerivedTime', 'DOUBLE', 'YES', None, None, None),
 ('TD_key0', 'DOUBLE', 'YES', None, None, None),
 ('TD_key2', 'DOUBLE', 'YES', None, None, None),
 ('TD_key3', 'DOUBLE', 'YES', None, None, None),
 ('Accel_XSamples', 'DOUBLE', 'YES', None, None, None),
 ('Accel_YSamples', 'DOUBLE', 'YES', None, None, None),
 ('Accel_ZSamples', 'DOUBLE', 'YES', None, None, None),
 ('Accel_samplerate', 'DOUBLE', 'YES', None, None, None),
 ('Power_IsPowerChannelOverrange', 'DOUBLE', 'YES', None, None, None),
 ('Power_Band1', 'DOUBLE', 'YES', None, None, None),
 ('Power_Band2', 'DOUBLE', 'YES', None, None, None),
 ('Power_Band3', 'DOUBLE', 'YES', None, None, None),
 ('Power_Band4', 'DOUBLE', 'YES', None, None, None),
 ('Power_Band5', 'DOUBLE', 'YES', None, None, None),
 ('Power_Band6', 'DOUBLE', 'YES', None, None, None),
 ('Power_Band7', 'DOUBLE', 'YES', None, None, None),
 ('Power_Band8', 'DOUBLE', 'YES', None, None, None),
 ('PO_infrared_filtered',

In [22]:
tbl = con.execute("SELECT * FROM overnight.r02L WHERE * LIKE 'Power_Band%' AND COLUMNS('Power_Band1') IS NOT NULL").df()

ParserException: Parser Error: syntax error at or near "*"
LINE 1: SELECT * FROM overnight.r02L WHERE * LIKE 'Power_Band%' AND COLUMNS('Power...
                                           ^

In [78]:
%%time
cols = ['DerivedTime', 'Power_Band1']
tbl = con.execute(f"select columns('^Power_Band|DerivedTime') from overnight.r02L where Power_Band1 is not null").df()

CPU times: user 4.61 s, sys: 37.1 ms, total: 4.65 s
Wall time: 4.65 s


In [79]:
%%time
cols = ['DerivedTime', 'Power_Band1']
tbl_2 = pl.DataFrame(con.execute(f"select columns('^Power_Band|DerivedTime') from overnight.r02L where Power_Band1 is not null").arrow())


CPU times: user 4.73 s, sys: 41.6 ms, total: 4.77 s
Wall time: 4.72 s


In [96]:
%%time
out = con.execute(f"select columns('DerivedTime|^TD_key') from overnight.r02L where TD_key0 is not null")
out.df().head()

CPU times: user 2.84 s, sys: 24.2 s, total: 27 s
Wall time: 27.3 s


,DerivedTime,TD_key0,TD_key2,TD_key3
0,1.650952e+12,-0.265827,0.015227,0.084200
1,1.650952e+12,0.210623,-0.016847,0.033114
2,1.650952e+12,0.281445,0.143844,0.181362
3,1.650952e+12,-0.203356,0.141900,0.192037
4,1.650952e+12,-0.206068,0.006048,0.064158


In [98]:
%%time
out = con.execute(f"select columns('DerivedTime|^TD_key') from overnight.r02L where TD_key0 is not null")
pl.DataFrame(out.arrow()).head()

CPU times: user 2.19 ms, sys: 0 ns, total: 2.19 ms
Wall time: 1.12 ms


In [111]:
con.execute("summarize overnight.r02L").arrow()

column_name: string
column_type: string
min: string
max: string
approx_unique: string
avg: string
std: string
q25: string
q50: string
q75: string
count: int64
null_percentage: string

In [107]:
%%time
command = """
select columns('DerivedTime|^TD_key')
from overnight.r02L
where @(TD_key0 - mean(TD_key0))/STDDEV(TD_key0) < 4 and TD_key0 is not null
"""
out = con.execute(command)
pl.DataFrame(out.arrow()).head()

BinderException: Binder Error: WHERE clause cannot contain aggregates!
LINE 4: where @(TD_key0 - mean(TD_key0))/STDDEV(TD_key0) < 4 and TD_key0 is not null
...
                          ^